<a href="https://colab.research.google.com/github/kospi-2025/EVT/blob/main/ADtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q lmoments3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd

In [2]:
base_url = "https://raw.githubusercontent.com/kospi-2025/EVT/main/"

df = pd.read_csv(base_url + "source_data/" + "ticker_info.csv")
df['id'] = df['id'].astype(str).str.zfill(6)
df['Yahoo_Ticker'] = df['id'] + ".KS"

ticker_to_name = dict(zip(df["Yahoo_Ticker"], df["name"]))
ticker_to_sector = dict(zip(df["Yahoo_Ticker"], df["sector"]))

In [3]:
logDD = pd.read_csv(base_url + "logDD.csv", index_col=0)
MRL = pd.read_csv(base_url + "MRL.csv")

In [19]:
import numpy as np
import pandas as pd
from scipy.stats import genpareto
import lmoments3 as lm

from lmoments3 import distr
from tqdm import tqdm

# Load MRL results (already computed)
mrl_df = MRL

# Function to compute Anderson-Darling statistic
def ad_statistic(data, cdf):
    data_sorted = np.sort(data)
    n = len(data_sorted)
    F = cdf(data_sorted)
    i = np.arange(1, n + 1)
    term = (2 * i - 1) * (np.log(F) + np.log(1 - F[::-1]))
    return -n - np.mean(term)

# Parameters
n_bootstrap = 100
min_exceedances = 10

# Results
results_mle = []
results_lmom = []

# Main loop
for _, row in tqdm(mrl_df.iterrows(), total=len(mrl_df)):
    ticker = row['Ticker']
    u = row['Threshold']

    # Replace with your actual logDD source
    series = logDD[ticker].dropna()
    excesses = series[series > u] - u

    if len(excesses) < min_exceedances:
        continue

    ## --- MLE-based GPD fit --- ##
    try:
        c_mle, loc_mle, scale_mle = genpareto.fit(excesses, floc=0)
        ad_emp_mle = ad_statistic(excesses, lambda x: genpareto.cdf(x, c_mle, loc=0, scale=scale_mle))

        ad_boot_mle = [
            ad_statistic(genpareto.rvs(c=c_mle, scale=scale_mle, size=len(excesses)),
                         lambda x: genpareto.cdf(x, c_mle, loc=0, scale=scale_mle))
            for _ in range(n_bootstrap)
        ]
        pval_mle = np.mean(np.array(ad_boot_mle) > ad_emp_mle)

        results_mle.append({
            "Ticker": ticker, "Threshold": u, "Quantile": row["Quantile"], "MRL": row["MRL"],
            "Method": "MLE", "AD_Stat": ad_emp_mle, "p_value": pval_mle,
            "xi_hat": c_mle, "beta_hat": scale_mle, "n_excess": len(excesses)
        })
    except Exception:
        continue

    ## --- L-moment-based GPD fit --- ##
    try:
        paras = distr.gpa.lmom_fit(excesses)
        gpa_fitted = distr.gpa(**paras)
        ad_emp_lmom = ad_statistic(excesses, gpa_fitted.cdf)

        ad_boot_lmom = [
            ad_statistic(gpa_fitted.rvs(size=len(excesses)), gpa_fitted.cdf)
            for _ in range(n_bootstrap)
        ]
        pval_lmom = np.mean(np.array(ad_boot_lmom) > ad_emp_lmom)

        results_lmom.append({
            "Ticker": ticker, "Threshold": u, "Quantile": row["Quantile"], "MRL": row["MRL"],
            "Method": "L-Moment", "AD_Stat": ad_emp_lmom, "p_value": pval_lmom,
            "xi_hat": paras["c"], "beta_hat": paras["scale"], "n_excess": len(excesses)
        })
    except Exception:
        continue

# Convert to DataFrames
df_mle = pd.DataFrame(results_mle)
df_lmom = pd.DataFrame(results_lmom)

# Save
df_mle.to_csv("ad_test_results_mle.csv", index=False)
df_lmom.to_csv("ad_test_results_lmoment.csv", index=False)

  0%|          | 0/19008 [00:00<?, ?it/s]<ipython-input-19-6afc6b4e0847>:18: RuntimeWarning: divide by zero encountered in log
  term = (2 * i - 1) * (np.log(F) + np.log(1 - F[::-1]))
100%|██████████| 19008/19008 [1:03:00<00:00,  5.03it/s]
